In [3]:
!pip install torch transformers evaluate rouge_score datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=11c26803b0197acf7c805354e2d3abd222900f5c71dadd6c7201684d37030f80
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [4]:
import torch
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, BertTokenizerFast
from transformers import get_linear_schedule_with_warmup, AdamW
import evaluate
import numpy as np
import pandas as pd
import json

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
with open('/datasets/cy_ml_resource/nlp-dataset/data-combine-v2.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)
    
df = pd.DataFrame(json_data['data'][0:546]).drop(labels='approved', axis=1)

dataset = Dataset.from_pandas(df)

cuda


In [ ]:
model_name = "fnlp/bart-large-chinese" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

max_input_length = 256
max_target_length = 48

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/259k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["content"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 應用前處理函數
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/546 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

metric = evaluate.load("rouge")

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() if isinstance(pred, str) else "" for pred in preds]
    labels = [label.strip() if isinstance(label, str) else "" for label in labels]

    preds = ["\n".join(pred.split("\n")[:1]) for pred in preds]
    labels = ["\n".join(label.split("\n")[:1]) for label in labels]

    return preds, labels

In [ ]:
training_args = TrainingArguments(
    output_dir="./model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    learning_rate=2.1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    num_train_epochs=60,
    fp16=torch.cuda.is_available(),
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,No log,0.995896
1,No log,0.720150
2,No log,0.464191
4,No log,0.164254
5,No log,0.103292
6,No log,0.049946
8,No log,0.055669
9,No log,0.052324
10,No log,0.048639
12,No log,0.010810


In [ ]:
trainer.save_model("model_export")